命名变量和张量

In [1]:
import tensorflow as tf

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
a = tf.constant(0.)
b = tf.constant(1.)
c = tf.constant(2.,name='cool_const')
d = tf.constant(3.,name='cool_const')
print(a.name,b.name,c.name,d.name)

虽然节点命名并非必要，但在调试时非常有用

使用范围

In [ ]:
a = tf.constant(0.)
b = tf.constant(1.)
with tf.variable_scope('first_scope'):
    c = a + b 
    d = tf.constant(2.,name='cool_const')
    coef1 = tf.get_variable('coef',[],initializer=tf.constant_initializer(2.))
    with tf.variable_scope('second_scope'):
        e = coef1 * d
        coef2 = tf.get_variable('coef',[],initializer=tf.constant_initializer(3.))
        f = tf.constant(1.)
        g = coef2 * f
print(a.name,b.name)
print(c.name,d.name)
print(e.name,f.name,g.name)
print(coef1.name)
print(coef2.name)

保存模型

In [ ]:
a1 = tf.get_variable('a1',[])
b1 = tf.get_variable('b1',[])
init = tf.global_variables_initializer()

saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.save(sess,'model/tftcp.model')

tftcp.model.data-00000-of-00001 包含模型权重（上述第一个要点）。它可能这里最大的文件。

tftcp.model.meta 是模型的网络结构（上述第二个要点）。它包含重建图形所需的所有信息。

tftcp.model.index 是连接前两点的索引结构。用于在数据文件中找到对应节点的参数。

checkpoint 实际上不需要重建模型，但如果在整个训练过程中保存了多个版本的模型，那它会跟踪所有内容。

加载模型

In [ ]:
a = tf.get_variable('a',[])
b = tf.get_variable('b',[])
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess,save_path='model/tftcp.model')
sess.run([a,b])

选择变量

In [ ]:
a = tf.get_variable('a', [])
b = tf.get_variable('b', [])
saver = tf.train.Saver()
c = tf.get_variable('c', [])
print(saver._var_list)

因为在创建 saver 时 c 还没有出现，所以它并没有成为函数的一部分。一般来说，你要在创建 saver 之前确保已经创建了所有的变量。

加载修正模型

首先，如果我们想保存一个完整的模型，但只想加载其中的一部分


In [ ]:
a = tf.get_variable('a', [])
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.restore(sess, './tftcp.model')
sess.run(a)

In [ ]:
a = tf.get_variable('a', [])
d = tf.get_variable('d', [])
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.restore(sess, './tftcp.model')

我们只想加载 a，却忽略了新变量 b。我们犯了一个错误，却抱怨 d 没有出现在 checkpoint 中。

第三种情况是，我们想将一个模型的参数加载到另一个模型的计算图中。这也会引发一个错误，原因很明显：Tensorflow 不知道把加载的所有参数放置在何处。幸好有个方法可以给它点提示。

In [ ]:
d = tf.get_variable('d', [])
init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list={'a': d})
sess = tf.Session()
sess.run(init)
saver.restore(sess, './tftcp.model')
sess.run(d)

这是一种关键机制，通过这个机制，可以将没有相同计算图的模型组合在一起。例如，你可能从网上获得了一个预训练好的语言模型，希望重用词嵌入。或者你可能在两次训练之间改变了模型的参数化，想让这个新版本在旧版本的基础上继续前进；但你又不想重新训练整个过程。在这两种情况下，你只需手动创建一个字典，将旧变量名称映射到新变量即可。



需要注意的是：你要明确地知道正在加载的参数是如何使用的。如果可以，你应该使用原作者用来构建模型的确切代码，以确保计算图的组件与训练时看起来一样。如果需要复现模型，务必记住，无论多微小的更改，都可能严重损害预训练网络的性能。所以始终要将复现结果和原来的结果进行对比。

模型检查

In [ ]:
a = tf.get_variable('a', [])
b = tf.get_variable('b', [10, 20])
c = tf.get_variable('c', [])
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.save(sess, './tftcp.model')
print(tf.contrib.framework.list_variables('./tftcp.model'))